## Загрузка данных <a name="data-loader"></a>

In [1]:
import numpy as np

In [2]:
# используем пакет https://pypi.org/project/rs-datasets/
!pip install rs_datasets

In [4]:
from rs_datasets import MovieLens

data = MovieLens("100k")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [5]:
users_count_from_log = data.ratings['user_id'].unique().shape 
items_count_from_log = data.ratings['item_id'].unique().shape

In [6]:
# загрузим данные в Spark

from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

## Сценарий с разными моделями

In [7]:
import numpy as np
import replay

## Пример с KNN

In [8]:
from replay.models import *

MODELS = {'knn_': KNN(), 
         'slim_': SLIM(),
         'random_rec_': RandomRec(),
         'als_': ALSWrap()}

PARAM_GRID = {'knn_':
              {
                  'num_neighbours': [3, 100],
                  'shrink': []
              },
              'slim_':
              {
                  'beta': [2**(-8), 2], 
                  'lambda_': [2**(-8), 2]
              },
              'random_rec_':
              {
                  'distribution': ['popular_based', 'uniform']
              },
              'als_':
              {
                  'rank': [8, 512]
              }
             }

/Users/a18785698/Documents/code_dir/venv376/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


##  Схема валидации

In [9]:
from replay.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=1,
    user_test_size=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
)

## Обучение сценария <a name="fit-scenario"></a>

In [10]:
from replay.scenarios import MainScenario
from replay.metrics import NDCG, HitRate

### KNN

In [11]:
k = 10
k_for_metrics = [10, 5, 1] 
budget = 20
metrics={
        NDCG: k_for_metrics,
        HitRate: k_for_metrics
}

In [12]:
scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=MODELS['knn_'],
    criterion=NDCG,
    metrics=metrics
)

In [13]:
best_knn_params = scenario.research(PARAM_GRID['knn_'],
    log,
    k=k,
    n_trials=budget
)
knn_results = scenario.experiment.results

08-Dec-20 20:44:23, replay, DEBUG: Деление лога на обучающую и тестовую выборку
DEBUG:replay:Деление лога на обучающую и тестовую выборку
08-Dec-20 20:44:27, replay, DEBUG: Длина трейна и теста: 99812 187
DEBUG:replay:Длина трейна и теста: 99812 187
08-Dec-20 20:44:28, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
DEBUG:replay:Количество пользователей в трейне и тесте: 943, 187
08-Dec-20 20:44:29, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 146
DEBUG:replay:Количество объектов в трейне и тесте: 1681, 146
08-Dec-20 20:44:29, replay, DEBUG: Инициализация метрик
DEBUG:replay:Инициализация метрик
08-Dec-20 20:44:29, replay, DEBUG: Обучение и предсказание дополнительной модели
DEBUG:replay:Обучение и предсказание дополнительной модели
08-Dec-20 20:44:29, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 20:44:29, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit

DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 20:47:05, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало обучения KNN
08-Dec-20 20:47:05, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 20:47:06, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 20:47:06, replay, DEBUG: Начало предикта KNN
DEBUG:replay:Начало предикта KNN
08-Dec-20 20:47:14, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 20:47:25, replay, DEBUG: NDCG=0.13
DEBUG:replay:NDCG=0.13
[I 2020-12-08 20:47:25,226] Trial 7 finished with value: 0.13245081058544958 and parameters: {'num_neighbours': 56, 'shrink': 72}. Best is trial 1 with value: 0.16839422507140372.
08-Dec-20 20:47:25, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 20:47:25, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало

DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 20:50:02, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 20:50:02, replay, DEBUG: Начало предикта KNN
DEBUG:replay:Начало предикта KNN
08-Dec-20 20:50:10, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 20:50:21, replay, DEBUG: NDCG=0.16
DEBUG:replay:NDCG=0.16
[I 2020-12-08 20:50:21,695] Trial 16 finished with value: 0.15632557775436584 and parameters: {'num_neighbours': 26, 'shrink': 38}. Best is trial 1 with value: 0.16839422507140372.
08-Dec-20 20:50:21, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 20:50:21, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало обучения KNN
08-Dec-20 20:50:21, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 20:50:22, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:repl

In [14]:
best_knn_params

{'num_neighbours': 9, 'shrink': 30}

### ALS

In [15]:
scenario.recommender=MODELS['als_']

best_als_params = scenario.research(PARAM_GRID['als_'],
    log,
    k=k,
    n_trials=budget
)
als_results = scenario.experiment.results

08-Dec-20 20:51:26, replay, DEBUG: Деление лога на обучающую и тестовую выборку
DEBUG:replay:Деление лога на обучающую и тестовую выборку
08-Dec-20 20:51:26, replay, DEBUG: Длина трейна и теста: 99812 187
DEBUG:replay:Длина трейна и теста: 99812 187
08-Dec-20 20:51:27, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
DEBUG:replay:Количество пользователей в трейне и тесте: 943, 187
08-Dec-20 20:51:27, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 146
DEBUG:replay:Количество объектов в трейне и тесте: 1681, 146
08-Dec-20 20:51:27, replay, DEBUG: Инициализация метрик
DEBUG:replay:Инициализация метрик
08-Dec-20 20:51:27, replay, DEBUG: Обучение и предсказание дополнительной модели
DEBUG:replay:Обучение и предсказание дополнительной модели
08-Dec-20 20:51:27, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 20:51:27, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit

DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 20:55:31, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало предикта ALSWrap
08-Dec-20 20:55:33, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 20:55:43, replay, DEBUG: NDCG=0.34
DEBUG:replay:NDCG=0.34
[I 2020-12-08 20:55:43,725] Trial 7 finished with value: 0.33794775856224774 and parameters: {'rank': 150}. Best is trial 2 with value: 0.36105657077102077.
08-Dec-20 20:55:43, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 20:55:43, replay, DEBUG: Начало обучения ALSWrap
DEBUG:replay:Начало обучения ALSWrap
08-Dec-20 20:55:43, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 20:55:47, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 20:55:47, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало преди

08-Dec-20 21:01:38, replay, DEBUG: NDCG=0.32
DEBUG:replay:NDCG=0.32
[I 2020-12-08 21:01:38,628] Trial 16 finished with value: 0.3186297479615757 and parameters: {'rank': 488}. Best is trial 2 with value: 0.36105657077102077.
08-Dec-20 21:01:38, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 21:01:38, replay, DEBUG: Начало обучения ALSWrap
DEBUG:replay:Начало обучения ALSWrap
08-Dec-20 21:01:38, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:01:41, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 21:01:41, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало предикта ALSWrap
08-Dec-20 21:01:44, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 21:01:55, replay, DEBUG: NDCG=0.32
DEBUG:replay:NDCG=0.32
[I 2020-12-08 21:01:55,116] Trial 17 finished with value: 0.3

In [16]:
all_results = knn_results.append(als_results)

In [17]:
best_als_params

{'rank': 67}

### SLIM

In [19]:
scenario.recommender=MODELS['slim_']

best_slim_params = scenario.research(PARAM_GRID['slim_'],
    log,
    k=k,
    n_trials=budget
)
slim_results = scenario.experiment.results

08-Dec-20 21:02:40, replay, DEBUG: Деление лога на обучающую и тестовую выборку
DEBUG:replay:Деление лога на обучающую и тестовую выборку
08-Dec-20 21:02:41, replay, DEBUG: Длина трейна и теста: 99812 187
DEBUG:replay:Длина трейна и теста: 99812 187
08-Dec-20 21:02:41, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
DEBUG:replay:Количество пользователей в трейне и тесте: 943, 187
08-Dec-20 21:02:41, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 146
DEBUG:replay:Количество объектов в трейне и тесте: 1681, 146
08-Dec-20 21:02:41, replay, DEBUG: Инициализация метрик
DEBUG:replay:Инициализация метрик
08-Dec-20 21:02:41, replay, DEBUG: Обучение и предсказание дополнительной модели
DEBUG:replay:Обучение и предсказание дополнительной модели
08-Dec-20 21:02:41, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 21:02:41, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit

08-Dec-20 21:04:43, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
08-Dec-20 21:04:43, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:04:43, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 21:04:43, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM
08-Dec-20 21:04:50, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 21:05:00, replay, DEBUG: NDCG=0.26
DEBUG:replay:NDCG=0.26
[I 2020-12-08 21:05:00,802] Trial 7 finished with value: 0.2596295767428304 and parameters: {'beta': 0.17364367775702533, 'lambda_': 0.0068053117999915}. Best is trial 6 with value: 0.2922760037502497.
08-Dec-20 21:05:00, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 21:05:00, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
08-Dec

DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 21:07:32, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
08-Dec-20 21:07:32, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:07:33, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
08-Dec-20 21:07:33, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM
08-Dec-20 21:07:40, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
08-Dec-20 21:07:50, replay, DEBUG: NDCG=0.26
DEBUG:replay:NDCG=0.26
[I 2020-12-08 21:07:50,914] Trial 16 finished with value: 0.2580769932071518 and parameters: {'beta': 0.0595148288348506, 'lambda_': 0.013637738784511147}. Best is trial 6 with value: 0.2922760037502497.
08-Dec-20 21:07:50, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
08-Dec-20 21:07:50, replay, DEBUG: Начало обуч

In [20]:
all_results = all_results.append(slim_results)

In [21]:
best_slim_params

{'beta': 0.009490422568703975, 'lambda_': 0.0605020866514255}

### RandomRec (чтобы проверить категориальный параметр)

In [22]:
scenario.recommender=MODELS['random_rec_']

best_rr_params = scenario.research(PARAM_GRID['random_rec_'],
    log,
    k=k,
    n_trials=2
)
rr_results = scenario.experiment.results

08-Dec-20 21:08:58, replay, DEBUG: Деление лога на обучающую и тестовую выборку
DEBUG:replay:Деление лога на обучающую и тестовую выборку
08-Dec-20 21:08:59, replay, DEBUG: Длина трейна и теста: 99812 187
DEBUG:replay:Длина трейна и теста: 99812 187
08-Dec-20 21:08:59, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
DEBUG:replay:Количество пользователей в трейне и тесте: 943, 187
08-Dec-20 21:08:59, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 146
DEBUG:replay:Количество объектов в трейне и тесте: 1681, 146
08-Dec-20 21:08:59, replay, DEBUG: Инициализация метрик
DEBUG:replay:Инициализация метрик
08-Dec-20 21:08:59, replay, DEBUG: Обучение и предсказание дополнительной модели
DEBUG:replay:Обучение и предсказание дополнительной модели
08-Dec-20 21:08:59, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 21:08:59, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit

In [23]:
all_results = all_results.append(rr_results)

In [24]:
best_rr_params

{'distribution': 'popular_based'}

## Результаты

In [25]:
all_results.sort_values('NDCG@10', ascending=False)

,HitRate@1,HitRate@5,HitRate@10,NDCG@10,NDCG@1,NDCG@5
ALSWrap{'rank': 67},0.192513,0.422460,0.588235,0.365985,0.192513,0.312600
ALSWrap{'rank': 41},0.181818,0.422460,0.609626,0.361057,0.181818,0.300219
ALSWrap{'rank': 66},0.192513,0.427807,0.582888,0.360516,0.192513,0.311691
ALSWrap{'rank': 62},0.187166,0.422460,0.577540,0.354988,0.187166,0.304955
ALSWrap{'rank': 44},0.187166,0.443850,0.550802,0.353707,0.187166,0.318901
ALSWrap{'rank': 88},0.197861,0.411765,0.556150,0.352575,0.197861,0.306384
ALSWrap{'rank': 35},0.160428,0.417112,0.598930,0.349640,0.160428,0.290681
ALSWrap{'rank': 90},0.192513,0.411765,0.550802,0.348166,0.192513,0.303574
ALSWrap{'rank': 129},0.176471,0.427807,0.545455,0.345485,0.176471,0.307595
ALSWrap{'rank': 150},0.176471,0.390374,0.545455,0.337948,0.176471,0.287519


## Проверка, что production работает ок

In [26]:
from replay.experiment import Experiment

In [27]:
train, test = user_random_splitter.split(log)
e = Experiment(test, {NDCG(): k_for_metrics, HitRate(): k_for_metrics})

### Предсказание с помощью сценария

In [28]:
best_slim_params

{'beta': 0.009490422568703975, 'lambda_': 0.0605020866514255}

In [29]:
scenario.recommender=MODELS['slim_']

recs = scenario.production(
    best_slim_params, 
    train,
    users=test.select("user_id").distinct().cache(), 
    items=test.select("item_id").distinct().cache(),
    k=10
)

e.add_result('best_slim_scenario', recs)

08-Dec-20 21:09:32, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
08-Dec-20 21:09:32, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
/Users/a18785698/Documents/code_dir/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
08-Dec-20 21:09:33, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM
08-Dec-20 21:09:34, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 21:09:34, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:09:35, replay, DEBUG: Начало предикта PopRec
DEBUG:replay:Начало предикта PopRec


In [30]:
scenario.recommender=MODELS['knn_']

recs = scenario.production(
    best_knn_params, 
    train,
    users=test.select("user_id").distinct().cache(), 
    items=test.select("item_id").distinct().cache(),
    k=10
)

e.add_result('best_knn_scenario', recs)

08-Dec-20 21:09:54, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало обучения KNN
08-Dec-20 21:09:54, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:09:55, replay, DEBUG: Начало предикта KNN
DEBUG:replay:Начало предикта KNN
08-Dec-20 21:09:56, replay, DEBUG: Начало обучения PopRec
DEBUG:replay:Начало обучения PopRec
08-Dec-20 21:09:56, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:09:57, replay, DEBUG: Начало предикта PopRec
DEBUG:replay:Начало предикта PopRec


In [31]:
all_results = all_results.append(e.results)

### Предсказание чистой моделью

In [32]:
best_slim_model = SLIM(**best_slim_params)
slim_pred = best_slim_model.fit_predict(train, 
                                        users=test.select("user_id").distinct().cache(),
                                        items=test.select("item_id").distinct().cache(),
                                        k=10)

08-Dec-20 21:10:31, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
08-Dec-20 21:10:31, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
08-Dec-20 21:10:32, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
08-Dec-20 21:10:32, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM


In [33]:
e.add_result('best_slim_model', slim_pred)

In [34]:
all_results.append(e.results).sort_values('NDCG@10', ascending=False)

,HitRate@1,HitRate@5,HitRate@10,NDCG@10,NDCG@1,NDCG@5
ALSWrap{'rank': 67},0.192513,0.422460,0.588235,0.365985,0.192513,0.312600
ALSWrap{'rank': 41},0.181818,0.422460,0.609626,0.361057,0.181818,0.300219
ALSWrap{'rank': 66},0.192513,0.427807,0.582888,0.360516,0.192513,0.311691
ALSWrap{'rank': 62},0.187166,0.422460,0.577540,0.354988,0.187166,0.304955
ALSWrap{'rank': 44},0.187166,0.443850,0.550802,0.353707,0.187166,0.318901
...,...,...,...,...,...,...
"KNN{'num_neighbours': 66, 'shrink': 69}",0.064171,0.155080,0.224599,0.136581,0.064171,0.114115
"KNN{'num_neighbours': 38, 'shrink': 96}",0.048128,0.176471,0.240642,0.134958,0.048128,0.114698
"KNN{'num_neighbours': 56, 'shrink': 72}",0.053476,0.155080,0.229947,0.132451,0.053476,0.108299
RandomRec{'distribution': 'popular_based'},0.010695,0.069519,0.101604,0.047353,0.010695,0.037040


Все ок, метрики при подборе параметров, применении модели в сценарии и просто обучении с лучшими гиперпараметрами совпадают.